In [ ]:
#default_exp layers

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Layers" data-toc-modified-id="Layers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Layers</a></span><ul class="toc-item"><li><span><a href="#Patches" data-toc-modified-id="Patches-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Patches</a></span><ul class="toc-item"><li><span><a href="#RoIHeads" data-toc-modified-id="RoIHeads-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>RoIHeads</a></span></li><li><span><a href="#GeneralizedRCNN" data-toc-modified-id="GeneralizedRCNN-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>GeneralizedRCNN</a></span></li></ul></li></ul></li><li><span><a href="#Export" data-toc-modified-id="Export-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Export</a></span></li></ul></div>

In [ ]:
#export
from mantisshrimp.imports import *
from mantisshrimp.core import *
from torchvision.models.detection.roi_heads import *

# Layers
> Definitions and patches of layers

## Patches

### RoIHeads

In [ ]:
# DEPRECATED
@patch
def forward(self:RoIHeads,
            features,      # type: Dict[str, Tensor]
            proposals,     # type: List[Tensor]
            image_shapes,  # type: List[Tuple[int, int]]
            targets=None   # type: Optional[List[Dict[str, Tensor]]]
            ):
    # type: (...) -> Tuple[List[Dict[str, Tensor]], Dict[str, Tensor]]
    """
    Arguments:
        features (List[Tensor])
        proposals (List[Tensor[N, 4]])
        image_shapes (List[Tuple[H, W]])
        targets (List[Dict])
    """
    if targets is not None:
        for t in targets:
            # TODO: https://github.com/pytorch/pytorch/issues/26731
            floating_point_types = (torch.float, torch.double, torch.half)
            assert t["boxes"].dtype in floating_point_types, 'target boxes must of float type'
            assert t["labels"].dtype == torch.int64, 'target labels must of int64 type'
            if self.has_keypoint():
                assert t["keypoints"].dtype == torch.float32, 'target keypoints must of float type'

    if not self.training:
        labels = None
        regression_targets = None
        matched_idxs = None
    if targets is not None:
        proposals, matched_idxs, labels, regression_targets = self.select_training_samples(proposals, targets)

    box_features = self.box_roi_pool(features, proposals, image_shapes)
    box_features = self.box_head(box_features)
    class_logits, box_regression = self.box_predictor(box_features)

    result = torch.jit.annotate(List[Dict[str, torch.Tensor]], [])
    losses = {}
    if targets is not None:
        assert labels is not None and regression_targets is not None
        loss_classifier, loss_box_reg = fastrcnn_loss(
            class_logits, box_regression, labels, regression_targets)
        losses = {
            "loss_classifier": loss_classifier,
            "loss_box_reg": loss_box_reg
        }
    if not self.training:
        boxes, scores, labels = self.postprocess_detections(class_logits, box_regression, proposals, image_shapes)
        num_images = len(boxes)
        for i in range(num_images):
            result.append(
                {
                    "boxes": boxes[i],
                    "labels": labels[i],
                    "scores": scores[i],
                }
            )

    if self.has_mask():
        mask_proposals = [p["boxes"] for p in result]
        if not self.training:
            pos_matched_idxs = None
        if targets is not None:
            assert matched_idxs is not None
            # during training, only focus on positive boxes
            num_images = len(proposals)
            mask_proposals = []
            pos_matched_idxs = []
            for img_id in range(num_images):
                pos = torch.nonzero(labels[img_id] > 0).squeeze(1)
                mask_proposals.append(proposals[img_id][pos])
                pos_matched_idxs.append(matched_idxs[img_id][pos])

        if self.mask_roi_pool is not None:
            mask_features = self.mask_roi_pool(features, mask_proposals, image_shapes)
            mask_features = self.mask_head(mask_features)
            mask_logits = self.mask_predictor(mask_features)
        else:
            mask_logits = torch.tensor(0)
            raise Exception("Expected mask_roi_pool to be not None")

        loss_mask = {}
        if targets is not None:
            assert targets is not None
            assert pos_matched_idxs is not None
            assert mask_logits is not None

            gt_masks = [t["masks"] for t in targets]
            gt_labels = [t["labels"] for t in targets]
            rcnn_loss_mask = maskrcnn_loss(
                mask_logits, mask_proposals,
                gt_masks, gt_labels, pos_matched_idxs)
            loss_mask = {
                "loss_mask": rcnn_loss_mask
            }
        if not self.training:
            labels = [r["labels"] for r in result]
            masks_probs = maskrcnn_inference(mask_logits, labels)
            for mask_prob, r in zip(masks_probs, result):
                r["masks"] = mask_prob

        losses.update(loss_mask)

    # keep none checks in if conditional so torchscript will conditionally
    # compile each branch
    if self.keypoint_roi_pool is not None and self.keypoint_head is not None \
            and self.keypoint_predictor is not None:
        keypoint_proposals = [p["boxes"] for p in result]
        if not self.training:
            pos_matched_idxs = None
        if targets is not None:
            # during training, only focus on positive boxes
            num_images = len(proposals)
            keypoint_proposals = []
            pos_matched_idxs = []
            assert matched_idxs is not None
            for img_id in range(num_images):
                pos = torch.nonzero(labels[img_id] > 0).squeeze(1)
                keypoint_proposals.append(proposals[img_id][pos])
                pos_matched_idxs.append(matched_idxs[img_id][pos])

        keypoint_features = self.keypoint_roi_pool(features, keypoint_proposals, image_shapes)
        keypoint_features = self.keypoint_head(keypoint_features)
        keypoint_logits = self.keypoint_predictor(keypoint_features)

        loss_keypoint = {}
        if targets is not None:
            assert targets is not None
            assert pos_matched_idxs is not None

            gt_keypoints = [t["keypoints"] for t in targets]
            rcnn_loss_keypoint = keypointrcnn_loss(
                keypoint_logits, keypoint_proposals,
                gt_keypoints, pos_matched_idxs)
            loss_keypoint = {
                "loss_keypoint": rcnn_loss_keypoint
            }
        if not self.training:
            assert keypoint_logits is not None
            assert keypoint_proposals is not None

            keypoints_probs, kp_scores = keypointrcnn_inference(keypoint_logits, keypoint_proposals)
            for keypoint_prob, kps, r in zip(keypoints_probs, kp_scores, result):
                r["keypoints"] = keypoint_prob
                r["keypoints_scores"] = kps

        losses.update(loss_keypoint)

    return result, losses

### GeneralizedRCNN

In [ ]:
# DEPRECATED
@torch.jit.unused
@patch
def eager_outputs(self:GeneralizedRCNN, losses, detections):
    # type: (Dict[str, Tensor], List[Dict[str, Tensor]]) -> Tuple[Dict[str, Tensor], List[Dict[str, Tensor]]]
    if losses and detections: return (losses, detections)
    if losses: return losses
    if detections: return detections

# Export


In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_data.core.ipynb.
Converted 04_data.annotations.ipynb.
Converted 06_data.load.ipynb.
Converted 07_transforms.ipynb.
Converted 08_models.ipynb.
Converted 11_metrics.core.ipynb.
Converted Untitled.ipynb.
Converted Untitled1.ipynb.
Converted data_refactor.ipynb.
Converted index.ipynb.
